<a href="https://colab.research.google.com/github/javierarteagapuell/deeplearning/blob/main/Prac2Ej1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Empezaremos obteniendo los itemsets frecuentes para k=1. En este punto necesitaremos obtener el
soporte de los itemset. Por lo tanto, se tendrá que usar un método que dado un itemset devuelva su
soporte.

LIBRERIAS mlxtend o apyori

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

#pip install mlxtend  

In [2]:
data = pd.read_csv('BlackFriday.csv', encoding='UTF-8')
data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [3]:
data.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3'],
      dtype='object')

In [4]:
data.isnull().values.any() 

True

Vemos en que columnas hay NA's

In [5]:
data.isna().sum()  

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2             72344
Product_Category_3            162562
dtype: int64

Como más de la mitad de las filas de Product Category 3 son NA's eliminamos esta columna

In [6]:
data = data.drop(['Product_Category_3'], axis=1)

Vemos si hay algún dato duplicado

In [7]:
if any(data.duplicated()):
  print("Hay datos duplicados")
else:
  print("No hay datos duplicados")

No hay datos duplicados


Agrupamos los productos por usuario

In [48]:
###columnas=['User_ID']
Server_ID = data.groupby('User_ID')['Product_ID'].apply(','.join)
Server_ID

#Server_ID = data.groupby('User_ID')['Product_ID'].apply(','.join).sum().unstack().reset_index().fillna(False).set_index('User_ID'))
#Server_ID

User_ID
1000001    P00259342,P00216342,P00157542,P00212942,P00070...
1000002    P00182742,P00102542,P00201442,P00303442,P00364...
1000003    P00025442,P00253142,P00319042,P00289942,P00148...
1000004                                            P00128942
1000005    P00081142,P00132642,P00223542,P00223642,P00108...
                                 ...                        
1006036    P00031042,P00162242,P00062942,P00112742,P00118...
1006037    P00058042,P00209842,P00115842,P00083842,P00059...
1006038    P00120542,P00316642,P00112142,P00131742,P00022...
1006039    P00291842,P00085842,P00019442,P00088242,P00188...
1006040    P00313342,P00179542,P00150042,P00129642,P00100...
Name: Product_ID, Length: 5891, dtype: object

In [44]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(Server_ID, min_support=0.06, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
frequent_itemsets.head()

AttributeError: ignored

Calculamos el soporte para K==1

In [ ]:
for i in range(0, len(data.User_ID)):
  num_Productos = data['Product_ID'].str.count(data.Product_ID[i]).sum()
  Soporte=num_Productos/len(data.User_ID)
  print(Soporte)

Itemset frecuentes (Son los que tengan un soporte alto)

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(Server_ID, min_support=0.06, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
frequent_itemsets.head()

In [ ]:
rules.head()

- ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos